### 安装项目所需的第三方库

In [ ]:
!pip install paddlepaddle==2.3.2 -i https://pypi.tuna.tsinghua.edu.cn/simple
!pip install parl -i https://pypi.tuna.tsinghua.edu.cn/simple
!pip install visualdl -i https://mirror.baidu.com/pypi/simple
!pip install gym==0.18.0 -i https://pypi.tuna.tsinghua.edu.cn/simple
!pip install tqdm -i https://pypi.tuna.tsinghua.edu.cn/simple
!pip install numpy -i https://pypi.tuna.tsinghua.edu.cn/simple

## PARL 强化学习 框架，架构图

![](https://ai-studio-static-online.cdn.bcebos.com/5cb6ac79d8ee4ad9b73d53063789c5c28d2ec36b499d47ef930ebf2facf64beb)


## train.py 代码及公式介绍

#### calc_reward_to_go 函数 介绍

In [5]:
# 基于 蒙特卡罗策略梯度，计算该episode每个步骤的未来总回报（奖励）
def calc_reward_to_go(reward_list, gamma=1.0):
    for i in range(len(reward_list) - 2, -1, -1):
        # G_i = r_i + γ·G_i+1
        reward_list[i] += gamma * reward_list[i + 1]  # Gt
    return np.array(reward_list)

该episode每个步骤的未来总回报（奖励）的计算公式如下：<br>
$\begin{equation}
\begin{split}
G_t &=\sum_{k=t+1}^{T} \gamma^{k-t-1}r_k\\ &= r_{t+1}+\gamma G_{t+1}
\end{split}
\end{equation}$

#### calc_reward_to_go 函数 举例

In [7]:
import numpy as np

# 创建 episode 回报（奖励）列表
# 代表该episode每个步骤的回报（奖励）
reward_list = [1, 2, 3, 4, 5, 6]

# 计算该episode每个步骤的未来总回报（奖励）
calc_reward_list = calc_reward_to_go(reward_list=reward_list, gamma=1.0)

print('该episode每个步骤的未来总回报（奖励）:\n', calc_reward_list)

该episode每个步骤的未来总回报（奖励）:
 [21. 20. 18. 15. 11.  6.]


其中: <br>
第一步的未来总回报（奖励）为 1+2+3+4+5+6 = 21 <br>
第二步的未来总回报（奖励）为 2+3+4+5+6 = 20 <br>
第三步的未来总回报（奖励）为 3+4+5+6 = 18 <br>
以此类推。。。。。 <br>

实现上，使用 动态规划 的思路
先 计算 第五步的未来总回报（奖励）:<br>
$\begin{equation}
\begin{split}
TotalReward_5=r_5+r_6=5+6=11
\end{split}
\end{equation}$

再 计算 第四步的未来总回报（奖励）:<br>
$\begin{equation}
\begin{split}
TotalReward_4&=r_4+r_5+r_6\\&=4+5+6\\&=r_4+TotalReward_5\\&=15
\end{split}
\end{equation}$<br>
以此类推。。。。。 <br>

## agent.py 代码及公式介绍

#### sample 函数 介绍

In [ ]:
# 根据输入观测，采样输出的离散动作，带探索
    def sample(self, obs):
        """ 根据观测值 obs 采样（带探索）一个动作
        """
        # 将 观测obs 转换为 Tensor张量
        obs = paddle.to_tensor(obs, dtype='float32')
        # 根据算法，返回每个离散动作选择的概率
        prob = self.alg.predict(obs)
        # 将 动作概率prob 转换为 Numpy数组
        prob = prob.numpy()

        # 根据动作概率选取动作
        # 从 离散动作中，根据 每个离散动作的动作概率prob，随机选取 1 个动作
        act = np.random.choice(len(prob), 1, p=prob)[0]
        return act

numpy.random.choice(a, size=None, replace=True, p=None)

用途：
从a(一维数据)中随机抽取数字，返回指定大小(size)的数组。
replace:True表示可以取相同数字，False表示不可以取相同数字。
数组p：与数组a相对应，表示取数组a中每个元素的概率，默认为选取每个元素的概率相同。

#### sample 函数 举例

In [16]:
import numpy as np

# 动作概率prob，假设有 3 个 离散动作
# 对应 每个动作 的 概率，概率总和 为 1.0
prob = np.array([0.4, 0.5, 0.1])

# 根据动作概率选取动作
# 从 离散动作中，根据 每个离散动作的动作概率prob，随机选取 1 个动作
act = np.random.choice(len(prob), 1, p=prob)[0]

print("随机选取到的动作为：", act)

随机选取到的动作为： 1


#### predict 函数 介绍

In [ ]:
# 根据输入观测，预测输出最优的离散动作
    def predict(self, obs):
        """ 根据观测值 obs 选择最优动作
        """
        # 将 观测obs 转换为 Tensor张量
        obs = paddle.to_tensor(obs, dtype='float32')
        # 根据算法，返回每个离散动作选择的概率
        prob = self.alg.predict(obs)
        # 根据动作概率选择概率最高的动作
        # 从 离散动作中，根据 每个离散动作的动作概率prob，选取概率最高的 1个 最优动作
        act = prob.argmax().numpy()[0]
        return act

paddle.argmax(x, axis=None, keepdim=False, dtype='int64', name=None) <br> 
<br>用途：
返回 沿 axis 计算输入 x 的最大元素的索引。
axis 默认值为None, 将会对输入的 x 进行平铺展开，返回最大值的索引。

#### predict 函数 举例

In [19]:
import paddle

# 动作概率prob，假设有 3 个 离散动作
# 对应 一个观测下，每个动作 的 概率，概率总和 为 1.0
prob = paddle.to_tensor([0.4, 0.5, 0.1], dtype='float32')

# 根据动作概率选取动作
# 从 离散动作中，根据 每个离散动作的动作概率prob，选取概率最高的 1个 最优动作
act = prob.argmax().numpy()[0]

print("选取到的最优动作为：", act)

选取到的最优动作为： 1


#### learn 函数 介绍

In [ ]:
# 智能体进行策略学习（回合更新）
    def learn(self, obs, act, reward):
        """ 根据训练数据更新一次模型参数
        """
        # act为动作数组，添加维度 ---> [act]
        act = np.expand_dims(act, axis=-1)
        # 回报（奖励）数组，添加维度 ---> [reward]
        reward = np.expand_dims(reward, axis=-1)

        # 将 观测obs数组 转换为 Tensor张量
        obs = paddle.to_tensor(obs, dtype='float32')
        # 将 动作act数组 转换为 Tensor张量，离散动作，以 整数 表示
        act = paddle.to_tensor(act, dtype='int32')
        # 将 回报（奖励）reward数组 转换为 Tensor张量
        reward = paddle.to_tensor(reward, dtype='float32')

        # 根据算法返回损失函数值
        loss = self.alg.learn(obs, act, reward)
        return loss.numpy()[0]

numpy.expand_dims(a, axis)

用途：
从a(数据)中插入一个新轴，该轴将出现在展开的数组形状中的轴位置。
轴axis：放置新轴（或多个轴）的扩展轴中的位置。

#### learn 函数 举例

In [25]:
import numpy as np

# 创建 act动作数组
act = np.array([0, 1, 2, 1, 1, 0])

# act为动作数组，添加维度 ---> [act]
act_new = np.expand_dims(act, axis=-1)

print('动作数组为：', act.shape)
print('添加维度后的动作数组为：\n', act_new)
print('添加维度后的动作数组维度为：\n', act_new.shape)

动作数组为： (6,)
添加维度后的动作数组为：
 [[0]
 [1]
 [2]
 [1]
 [1]
 [0]]
添加维度后的动作数组维度为：
 (6, 1)


## algorithm.py 代码及公式介绍

#### learn 函数 介绍

In [ ]:
# 智能体进行策略学习（回合更新）
    def learn(self, obs, action, reward):
        """ 用policy gradient 算法更新policy model
        """
        # 输入 观测obs张量（Tensor）到 神经网络模型（策略）
        # 获取神经网络（策略）输出的 动作概率prob张量（Tensor）
        prob = self.model(obs)

        # paddle.squeeze将输入Tensor的Shape中尺寸为1的维度进行压缩
        # 返回对维度进行压缩后的Tensor，数据类型与输入Tensor一致
        action_onehot = paddle.squeeze(

            # 将动作action 转换为 独热编码（one-hot encoding）
            # 其中 num_classes 用于定义一个one-hot向量的长度
            F.one_hot(action, num_classes=prob.shape[1]),

            # axis 代表要压缩的轴，默认为None，表示对所有尺寸为1的维度进行压缩。
            # 这边 输入 axis=1，表示对第1维数据维度为1的话，将进行压缩。
            axis=1)

        # 动作对数概率 = sum（对数动作概率（同策略神经网络输出的） * 动作的独热编码（one-hot encoding））
        # 求和运算sum的维度 axis=-1，代表最后一维
        log_prob = paddle.sum(paddle.log(prob) * action_onehot, axis=-1)

        # 对 reward张量（Tensor）第1维数据维度为1的话，将进行压缩。
        reward = paddle.squeeze(reward, axis=1)

        # 对 每个动作对数概率 * 对应动作的未来累计总回报（奖励）
        # 目标是 训练 使得 智能体的策略 能够 实现 每一步动作的未来累计总回报（奖励）最大化
        # 而 Adam 神经网络参数优化器 是 梯度下降
        # (-1 * ) 操作来实现 梯度上升
        loss = paddle.mean(-1 * log_prob * reward)

        # 清空梯度
        self.optimizer.clear_grad()
        # 反向传播
        loss.backward()
        # 参数更新
        self.optimizer.step()
        return loss

paddle.squeeze(x, axis=None, name=None)

用途：
会删除输入Tensor的Shape中尺寸为1的维度。如果指定了axis，则会删除axis中指定的尺寸为1的维度。如果没有指定axis，那么所有等于1的维度都会被删除。


paddle.nn.functional.one_hot(x, num_classes, name=None)
<br>
用途：
将输入'x'中的每个id转换为一个one-hot向量，其长度为 num_classes ，该id对应的向量维度上的值为1，其余维度的值为0。

#### learn 函数 举例

In [32]:
import paddle
import paddle.nn.functional as F

# 动作概率prob，假设有 3 个 离散动作
# 对应 在3个观测下，每个动作 的 概率，概率总和 为 1.0
prob = paddle.to_tensor(
    [
        [0.4, 0.5, 0.1], # obs1
        [0.2, 0.3, 0.5], # obs2
        [0.6, 0.2, 0.2]  # obs3
    ],
    dtype='float32'
)

# 动作action，假设有 3 个 离散动作
# 创建 action 动作数组，表示 在3个观测下，智能体 输出的 动作
action = np.array([2, 0, 2])
# 添加维度 ---> [action]
action = np.expand_dims(action, axis=-1)
# 将 动作action数组 转换为 Tensor张量，离散动作，以 整数 表示
action = paddle.to_tensor(action, dtype='int32')

# 将动作action 转换为 独热编码（one-hot encoding）
# 其中 num_classes 用于定义一个one-hot向量的长度
act_one_hot_encoding = F.one_hot(action, num_classes=prob.shape[1])
print('动作张量的独热编码的维度为：', act_one_hot_encoding.shape)
print('动作张量的独热编码为：', act_one_hot_encoding)

# 将动作张量的独热编码Shape中第1维数据维度为1的话，将进行压缩。
action_onehot = paddle.squeeze(act_one_hot_encoding, axis=1)
print('\n压缩后动作张量的独热编码的维度为：', action_onehot.shape)

动作张量的独热编码的维度为： [3, 1, 3]
动作张量的独热编码为： Tensor(shape=[3, 1, 3], dtype=float32, place=Place(cpu), stop_gradient=True,
       [[[0., 0., 1.]],

        [[1., 0., 0.]],

        [[0., 0., 1.]]])

压缩后动作张量的独热编码的维度为： [3, 3]


#### learn 函数 对应公式

代码：
> log_prob = paddle.sum(paddle.log(prob) * action_onehot, axis=-1)

对应公式：<br>
$\begin{equation}
\begin{split}
\nabla  logp_\theta(\tau )&=\nabla\sum_{t=1}^{T} logp_\theta(a_t|s_t)\\&=\sum_{t=1}^{T}\nabla logp_\theta(a_t|s_t)
\end{split}
\end{equation}$

代码：

> loss = paddle.mean(-1 * log_prob * reward)

对应公式：
$\begin{equation}
\begin{split}
\nabla \bar{R_\theta } = \frac{1}{T} \sum_{t=1}^{T} logp_\theta (a_t|s_t) R(\tau )
\end{split}
\end{equation}$

意思为：一个episode的轨迹内，平均每个步骤 **期望** 的未来累计总回报（奖励）。

为使得智能体决策的每个步骤 **期望** 的未来累计总回报（奖励）最大化（优化目标）。<br>使用梯度上升。<br>对应公式：<br>$\begin{equation}
\begin{split}
Loss&=-1*\nabla \bar{R_\theta } \\&= -1*\frac{1}{T} \sum_{t=1}^{T} logp_\theta (a_t|s_t) R(\tau )
\end{split}
\end{equation}$


## 参考链接
- [Paddle Documentation](https://www.paddlepaddle.org.cn/documentation/docs/zh/api/index_cn.html)